In [1]:
import tensorflow as tf
from keras.models import load_model
from keras.layers import *
from keras import Model, Sequential, Input
from keras import optimizers
import keras.backend as K
import numpy as np
from pprint import pprint
import pickle

/home/henrik/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def make_style_model():
    model = load_model('darknet.h5')
    inputs = model.layers[0].input
    x = model.layers[16].output
    for layer in model.layers:
        layer.trainable = False
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    return Model(inputs, x)


def EuclSqDist(inputs):
    return K.mean(K.square(inputs[0]- inputs[1]), axis=-1)


def EuclSqLambda():
    return Lambda(EuclSqDist, output_shape=(1,))

def contrastive_loss(y, d):
    """ Contrastive loss from Hadsell-et-al.'06
        http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    """
    margin = 1
    return K.mean(K.square(y * d + (1 - y) * (margin - d)))

def load(filepath):
    with open(filepath, 'rb') as f:
        return pickle.load(f)

In [3]:
base_model = make_style_model()
input_a = Input((256,256,3))
input_b = Input((256,256,3))
encoding_a = base_model(input_a)
encoding_b = base_model(input_b)
dist = EuclSqLambda()([encoding_a, encoding_b])
model = Model([input_a, input_b], dist)
model.summary()

/home/henrik/miniconda3/lib/python3.6/site-packages/keras/models.py:255: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 128)          4309104     input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 1)            0           model_1[1][0]                    
          

In [4]:
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss=contrastive_loss)
data = load('obj/imagePairs.pkl')
pairs = (data['pairs'] - 128) / 256
labels = data['labels']
model.fit([pairs[:,0], pairs[:,1]], labels, epochs=5)

Epoch 1/5
400/400 [==============================] - 52s 131ms/step - loss: 0.3913
Epoch 2/5
400/400 [==============================] - 55s 137ms/step - loss: 0.3941
Epoch 3/5
400/400 [==============================] - 56s 140ms/step - loss: 0.4246
Epoch 4/5
400/400 [==============================] - 55s 138ms/step - loss: 0.3863
Epoch 5/5
400/400 [==============================] - 55s 138ms/step - loss: 0.3731


In [5]:
def accuracy(pred, gt):
    return gt[pred.ravel() < 0.5].mean()

pred = model.predict([pairs[:, 0], pairs[:, 1]])
print(accuracy(pred, labels))

0.4984025559105431
